Refactoring lost histogram processing file - IT IS RECOVERED!!!

load all packages
load up IA file as dataframe
load up joined shapefile with census info
loop trhough each row, write to a new file, new headers, as well as old, for each new variable create it by saying if a value of a row's column so fow[23] for example is great than or equal to xyz then the new variable is true or false respectively, add new variable value to row, then write all to new csv file
drop unneeded columns
dg loc so its only 50 rows
change price to be a float and without funny symbols
create new variables (that you'll find listed out on the csv file from yesterday)

when you have the new variable in the new csv file, lets load that into qgis and join it again to the census shapefile, watch filenames
then we'll have a new variables, with census data, geolocated file! all in one place! 
then drop more columns that don't need (from census)

In [1]:
import geojson
import geopandas
import pandas as pd
import numpy as np
import statistics as st
import matplotlib as plt
import scipy
from scipy import stats
from scipy.stats import describe
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import seaborn as sns
import folium
import csv
from IPython.display import display, HTML
from pathlib import Path 
import os
from datetime import date, time, datetime
from dateutil import tz #for dealing with timezone
from geopy import distance
from django.contrib.gis.geos import GEOSGeometry, Point

/Users/stateofplace/.virtualenvs/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
def get_time():
    today = date.today()
    current_time = time(datetime.now().hour, datetime.now().minute, datetime.now().second)
    date_today = datetime.combine(today, current_time)
    new_date_time = str(date_today).replace(" ", "_").replace("-", "_").replace(":", "_")
    return(new_date_time)
print(get_time())
new_date_time = get_time()

2022_05_14_19_13_43


In [3]:
def load_files(ia, ia_old, census_ia): 
    #load and create dataframes
    ia_df = pd.read_csv(ia)
    ia_df_old = pd.read_csv(ia_old)
    #drop
    ia_df.drop(columns=["scrape_id", "bathrooms_text","host_verifications", "host_neighbourhood","property_type", "amenities", "host_verifications", "neighbourhood", "host_listings_count", "picture_url","host_url", "description", "neighborhood_overview", "host_about", "host_response_time", "host_response_rate", 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_has_profile_pic', 'host_identity_verified', 'calendar_updated', 'has_availability', 'calendar_last_scraped', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'neighbourhood_group_cleansed', 'beds', 'maximum_nights_avg_ntm', 'bathrooms', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], inplace=True)
    #rename columns ? 
    #trim columns 
    #ia_df = ia_df.loc[:50, :]
    ia_df_old.drop(columns=["scrape_id", "bathrooms_text","host_verifications", "host_neighbourhood","property_type", "amenities", "host_verifications", "neighbourhood", "host_listings_count", "picture_url","host_url", "description", "neighborhood_overview", "host_about", "host_response_time", "host_response_rate", 'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url', 'host_picture_url', 'host_has_profile_pic', 'host_identity_verified', 'calendar_updated', 'has_availability', 'calendar_last_scraped', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'minimum_nights', 'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'neighbourhood_group_cleansed', 'beds','maximum_nights_avg_ntm', 'bathrooms', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms'], inplace=True)

    #handle shape file joined, we need it for some variable creation, but doens't need to be a airbnb joined one, just the census info is fine
    census_gdf = geopandas.read_file(census_ia)
    return(ia_df, ia_df_old, census_gdf)
#ia_df_clean_old, census_gdf = load_files("Out_CSV/listings_9_1_2020_prepand.csv", "../thesis_app/airbnb_census_joined.shp")
ia_df_clean, ia_df_clean_old, census_gdf = load_files("../scrap_work/florence_listings_jan_27.csv", "Out_CSV/listings_9_1_2020_prepand.csv", "../thesis_app/airbnb_census_joined.shp") #using the shapefile with no airbnb data yet makes more sense

In [4]:
#will only work once i think
def fixpricing_file(dataframe):
    #price adjust
    dataframe['price'] = dataframe['price'].str.replace("$", "").str.replace(',', '')
    dataframe['price'] = pd.to_numeric(dataframe['price'])
    #tests
    # print(dataframe['price']) #no special characters 
    # print(type(dataframe['price'])) #float not string 
    return dataframe

ia_df_clean = fixpricing_file(ia_df_clean) #ia_df_clean 


/var/folders/9h/m6366gw17qd8jtbtczp7ttnc0000gp/T/ipykernel_7429/1573565530.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dataframe['price'] = dataframe['price'].str.replace("$", "").str.replace(',', '')


In [7]:
def df_to_csv(dataframe):
    filepath = Path('../thesis_app/Out_CSV/ia_1_27_22' + new_date_time + '.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True) 
    dataframe.to_csv(filepath)
    #tests
    print(filepath)
    return csv
df_to_csv(ia_df_clean)   #ia_df_clean ia_df_clean_old

../thesis_app/Out_CSV/ia_1_27_222022_05_14_19_13_43.csv


<module 'csv' from '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/csv.py'>

In [14]:
# ia_df_clean_old.head()
# #s.loc[s['has_liscense'] == 0]
# row_old_df = ia_df_clean_old.loc[ia_df_clean_old['id'] == 24469]

# row_old_df['host_id']
ia_df_clean.head()
ia_df_clean['minimum_nights_avg_ntm']


0        2.0
1        2.0
2        2.0
3        2.0
4        1.0
        ... 
10263    2.0
10264    3.0
10265    2.0
10266    3.0
10267    3.0
Name: minimum_nights_avg_ntm, Length: 10268, dtype: float64

In [59]:

def create_new_variables(cleaned_file_input, file_output, dataframe, dataframe_old):
    #do a lot of processing
    all = []
    with open(cleaned_file_input, 'r') as file:
        with open(file_output, 'w', encoding="utf-8") as newfile:
            csvwriter = csv.writer(newfile,lineterminator='\n')
            csvreader = csv.reader(file)
            #extract headers
            header = next(csvreader)
            new_headers = ['days_rented_ltm', 'rounded_revenue_ltm', 'occupancy_rate_approx', 'is_hotel', 'is_entire', 'many_listings', 'only_1_listing', 'only_2_listings', 'host_florence', 'has_liscense', 'is_instant_bookable', 'dist_duomo', 'is_centro', 'is_gavinana', 'is_isolotto', 'is_rifredi', 'is_campo', 'listing_revenue_exceed_LTR', 'effected_by_policy_1', 'effected_by_policy_2', 'effected_by_policy_3', 'commercial', 'geom']
            for item in new_headers:
                header.append(item)
            all.append(header)
            for row in csvreader:
                #index row 
                #TODO do i have togo by row number or can i use the title? like dg[host_location] not row[8] can i do row[host_location]
                listing_row_id = int(row[0])
                listing_id = int(row[1])
                #year and month of data scraped
                date_scraped = row[3]
                year = date_scraped.split('-')[0]
                month = date_scraped.split('-')[1]
                day = date_scraped.split('-')[2]
                #year started hosting 
                year_started_hosting = int(row[7].split('-')[0])
                month_started = int(row[7].split('-')[1])
                # total global and local listings
                #TODO: check if this is accurate local v global
                # global_total_listings = ia_df_clean['host_total_listings_count'] + ia_df_clean['calculated_host_listings_count'] #int(row[9]) + int(row[25]) not great to use df because then my value is a series! 
                #dist_duomo
                duomo_coords = (43.7731, 11.2560)
                #listing_coords = lat, long
                listing_coords = (row[11], row[12]) # ia_df_clean['latitude'], ia_df_clean['longitude']
                ##dist_duomo = duomo_coords - listing_coords
                dist_duomo = (distance.distance(duomo_coords, listing_coords).m)
                # geom = ((row[11], row[12]), srid=4326)
                # pnt = GEOSGeometry('POINT(5 23)') # WKT
                coords_wkt = (round(float(row[12]),4) , round(float(row[11]),4) )
                coords_wkt = Point(coords_wkt)
                # str(coords_wkt)
                # print(coords_wkt)
                # print(type(coords_wkt))
                geom = coords_wkt
                # geom = GEOSGeometry(f'POINT(round(float(row[11]),4) round(float(row[12]),4))')
                # geom = (row[11], row[12], 4326)
                # geom = (round(float(row[11]),4), round(float(row[12]),4))
                # geom = round(geom, 4)
                
                #has_liscense
                if len(ia_df_clean['license']) > 0: # row[23]
                    has_liscense = 1
                else:
                    has_liscense = 0
                #is_instant_bookable
                if row[26] == 't': # row[24]
                    is_instant_bookable = 1
                else:
                    is_instant_bookable = 0
                #getting info for "host_florence" success
                if ("Florence" or "Firenze") in ia_df_clean['host_location']: #row[8]
                    host_florence = 1
                else:
                    host_florence = 0
                    #print(host_florence)
                #getting info for "is_centro" success
                #got better average rent from statistica and square meters for rent burden
                centro_neighborhood_rent = 1238.70 * 12
                campo_neighborhood_rent = 1078.00 * 12
                rif_neighborhood_rent = 954.80 * 12
                iso_neighborhood_rent = 939.40 * 12
                gav_neighborhood_rent = 1070.30 * 12
                if "Centro" in row[10]: #row[10] ia_df_clean['neighbourhood_cleansed']
                    is_centro = 1
                else:
                    is_centro = 0
                #is_campo success
                if "Campo" in row[10]:
                    is_campo = 1
                else:
                    is_campo = 0
                #is_rifredi success
                if "Rifredi" in row[10]:
                    is_rifredi = 1
                else:
                    is_rifredi = 0
                if "Isolotto" in row[10]:
                    is_isolotto = 1
                else:
                    is_isolotto = 0
                if "Gavinana" in row[10]:
                    is_gavinana = 1
                else:
                    is_gavinana = 0
               #occupancy_rate_approx
                #did reviews a year divided by how likely someone gives a review (airbnb data) * avg stay in florence (2.6)
                review_rate = .50
                avg_length = 3.2 
                min_night_avg = float(row[18]) #minimum_nights_avg_ntm
                if len(row[23]) > 0:
                    # print(row[23])
                    num_reviews = float(row[23]) #number_of_reviews_ltm
                    if min_night_avg > avg_length:
                        avg_days = min_night_avg
                    else:
                        avg_days = avg_length
                    num_bookings = review_rate * num_reviews
                    days_rented_ltm = num_bookings * avg_days
                    # print(days_rented_ltm)
                    # days_rented = (int(row[22])/.7)*2.6 #old method
                    # if old_listing:
                    # if (year_started_hosting < 2019):
                    #     #couldve been more exact like 2020 and may but too complicated ... 
                    #     old_df_ltm_row = dataframe_old.loc[dataframe_old['id'] == listing_id]
                    #     #listing_id, 'number_of_reviews_ltm']
                    #     old_df_ltm = old_df_ltm_row['number_of_reviews_ltm']
                    #     old_df_ltm = pd.to_numeric(old_df_ltm_row['number_of_reviews_ltm'])
                    #     days_rented_prepan = ((old_df_ltm)*review_rate)*avg_days #load csv file here from 9/1/2020 because if all are started at least in 2018 then they'd be in there in 2020 pre pand, bit of a buffer... also will use dataframe and look up using .loc with row being the listing id of this and column being the review
                    #     days_rented_postpan = (int(row[22])*review_rate)*avg_days #why do i use total number rveiws not in last twelve months?  TODO: check that is right to use row 22 not 23
                    #     days_rented = (days_rented_prepan + days_rented_postpan)/2
                    #     days_rented_pan_adj = pd.to_numeric(days_rented)
                    # #     #do avg of pre covid post covid data based on yearly review
                    # # if new_listing:
                    # elif year_started_hosting >= 2019:
                    #     #new listing 
                    #     # pass
                    #     # #use monthly review
                    #     days_rented_pan_adj = (int(row[23])*review_rate)*avg_days #here i use revies from last twelve months 
                    # print(days_rented_pan_adj)
                    #for now... making it more conservative with the venice lady's .5, and changing to 3.2 based on statistica 
                    #days_rented = (int(row[21])/.5)*3.2
                    occupancy_rate_approx = (days_rented_ltm/365) * 100
                    if occupancy_rate_approx > 70.00:
                        occupancy_rate_approx = 70.00
                    # print(occupancy_rate_approx)
                    # occupancy_rate_approx_pan_adj = (days_rented_pan_adj/365) * 100
                    # print(occupancy_rate_approx_pan_adj)
                    #thought I handeled this in the price fix def
                    #revenue #have to strip the string for price then turn into float
                    price_stripped_dollar = row[16].strip('$')
                    price_replaced_comma = price_stripped_dollar.replace(',','')
                    new_price = float(price_replaced_comma)

                    #yearly revenue
                    # revenue_pan_adj = days_rented_pan_adj * new_price
                    # rounded_revenue_pan_adj = round(revenue_pan_adj, 2)
                    # print(rounded_revenue_pan_adj)
                    revenue_ltm = days_rented_ltm * new_price
                    rounded_revenue_ltm = round(revenue_ltm, 2)
                    # print(rounded_revenue_ltm)
                else:
                    days_rented_ltm = 'n/a'
                    rounded_revenue_ltm = 'n/a'
                    # listing_revenue_exceed_LTR = 'n/a'

                # getting if revenue from a listing exceeds approximiate LTR rent for the year issue wiht variable being assigned too early not sure
                if is_centro == 1:
                    if rounded_revenue_ltm == 'n/a':
                        listing_revenue_exceed_LTR = 'n/a'
                    elif rounded_revenue_ltm > centro_neighborhood_rent:
                        listing_revenue_exceed_LTR = 1
                    else:
                        listing_revenue_exceed_LTR = 0
                elif is_gavinana == 1:
                    if rounded_revenue_ltm == 'n/a':
                        listing_revenue_exceed_LTR = 'n/a'
                    elif rounded_revenue_ltm > gav_neighborhood_rent:
                        listing_revenue_exceed_LTR = 1
                    else:
                        listing_revenue_exceed_LTR = 0
                elif is_isolotto == 1:
                    if rounded_revenue_ltm == 'n/a':
                        listing_revenue_exceed_LTR = 'n/a'
                    elif rounded_revenue_ltm > iso_neighborhood_rent:
                        listing_revenue_exceed_LTR = 1
                    else:
                        listing_revenue_exceed_LTR = 0
                elif is_rifredi ==1:
                    if rounded_revenue_ltm == 'n/a':
                        listing_revenue_exceed_LTR = 'n/a'
                    elif rounded_revenue_ltm > rif_neighborhood_rent:
                        listing_revenue_exceed_LTR = 1
                    else:
                        listing_revenue_exceed_LTR = 0
                elif is_campo == 1:
                    if rounded_revenue_ltm == 'n/a':
                        listing_revenue_exceed_LTR = 'n/a'
                    elif rounded_revenue_ltm > campo_neighborhood_rent:
                        listing_revenue_exceed_LTR = 1
                    else:
                        listing_revenue_exceed_LTR = 0
                # print(listing_revenue_exceed_LTR)
                # listing_revenue_exceed_LTR = 'n/a'    
                #many_listings variable success
                if  int(row[9]) > 2: # ia_df_clean['host_total_listings_count']
                    many_listings = 1
                else:
                    many_listings = 0
                #only_1 success
                if int(row[9]) == 1:
                    only_1_listing = 1
                else: 
                    only_1_listing = 0
                #only_2 success
                if int(row[9]) == 2:
                    only_2_listings = 1
                else:
                    only_2_listings = 0
                #is_entire success 
                if row[13] == 'Entire home/apt':  # ia_df_clean['room_type']
                    is_entire = 1
                else:
                    is_entire = 0
                #is_hotel success
                if row[13] == 'Hotel room':
                    is_hotel = 1
                else:
                    is_hotel = 0
        
                #commercial
                if dataframe.loc[listing_row_id, 'host_total_listings_count'] > 2:
                    commercial = 1
                elif host_florence == 0:
                    commercial = 1
                else:
                    commercial = 0
                #very_likely_commercial
                if dataframe.loc[listing_row_id, 'host_total_listings_count'] > 2 and dataframe.loc[listing_row_id, 'instant_bookable'] == 1:
                    very_likely_commercial = 1
                else:
                    very_likely_commercial = 0

                #effected_by_policy_x (1-liscense,2- -is entire, 3-2 listing max, 4-day limit 90) #switched 2 and 4 from origingal 
                if has_liscense == 0:
                    effected_by_policy_1 = 1
                else: 
                    effected_by_policy_1 = 0
                if is_entire == 1:
                    effected_by_policy_2 = 1
                else:
                    effected_by_policy_2 = 0
                #man_listings is more than 2 
                if many_listings == 1:
                    effected_by_policy_3 = 1
                else:
                    effected_by_policy_3 = 0
                print(geom)
                new_variables = [days_rented_ltm, rounded_revenue_ltm, occupancy_rate_approx, is_hotel, is_entire, many_listings, only_1_listing, only_2_listings, host_florence, has_liscense, is_instant_bookable, dist_duomo, is_centro, is_gavinana, is_isolotto, is_rifredi, is_campo, listing_revenue_exceed_LTR, effected_by_policy_1, effected_by_policy_2, effected_by_policy_3, commercial, geom]
                for item in new_variables:
                    row.append(item)
                #adding entire row to all 
                all.append(row)
            #writing all rows to new out file
            csvwriter.writerows(all)
    return csvwriter
create_new_variables('../thesis_app/Out_CSV/ia_1_27_222022_05_14_19_13_43.csv', '../thesis_app/Out_CSV/ia_1_27_222022_05_14_19_13_43_new_variables_encode_test.csv', ia_df_clean, ia_df_clean_old)

POINT (11.2439 43.7821)
POINT (11.244 43.782)
POINT (11.244 43.782)
POINT (11.244 43.782)
POINT (11.2575 43.7702)
POINT (11.2522 43.7771)
POINT (11.2774 43.7616)
POINT (11.2435 43.7683)
POINT (11.2415 43.7627)
POINT (11.2526 43.7752)
POINT (11.2526 43.7752)
POINT (11.2438 43.7674)
POINT (11.2633 43.7694)
POINT (11.2633 43.7694)
POINT (11.2633 43.7694)
POINT (11.2462 43.7642)
POINT (11.2375 43.7728)
POINT (11.2522 43.777)
POINT (11.2522 43.777)
POINT (11.2522 43.777)
POINT (11.2503 43.7739)
POINT (11.2526 43.7752)
POINT (11.2364 43.7717)
POINT (11.2382 43.7716)
POINT (11.2479 43.7724)
POINT (11.2642 43.7824)
POINT (11.2618 43.7771)
POINT (11.2507 43.7673)
POINT (11.2559 43.7765)
POINT (11.2632 43.7726)
POINT (11.2647 43.7821)
POINT (11.2597 43.7709)
POINT (11.2686 43.7826)
POINT (11.2594 43.7715)
POINT (11.2672 43.7706)
POINT (11.2632 43.7679)
POINT (11.2585 43.7759)
POINT (11.2526 43.7752)
POINT (11.2581 43.7793)
POINT (11.2503 43.776)
POINT (11.2526 43.7752)
POINT (11.2526 43.7752)
PO

In [60]:
# df = pd.read_csv("../thesis_app/Out_CSV/ia_1_27_222022_05_14_19_13_43_new_variables_encode_test.csv")
df = pd.read_csv("../thesis_app/Out_CSV/ia_1_27_222022_05_14_19_13_43_new_variables_encode_test.csv")
df.drop(columns=["Unnamed: 0","name", "host_name","host_location"], inplace=True)




In [61]:
df.to_csv("../../geodjango_tut/geodjango/world/data/csv_ia/to_migrate_ia_1_27_222022_05_14_19_13_43_new_variables_noname_dropped_loc_encode_geom_point.csv")

In [62]:
df.head()

,id,listing_url,last_scraped,host_id,host_since,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,room_type,...,is_gavinana,is_isolotto,is_rifredi,is_campo,listing_revenue_exceed_LTR,effected_by_policy_1,effected_by_policy_2,effected_by_policy_3,commercial,geom
0,24469,https://www.airbnb.com/rooms/24469,2021-12-17,99178,2010-03-26,5,Centro Storico,43.78210,11.24392,Entire home/apt,...,0,0,0,0,0,0,1,1,1,POINT (11.2439 43.7821)
1,24470,https://www.airbnb.com/rooms/24470,2021-12-17,99178,2010-03-26,5,Centro Storico,43.78202,11.24399,Entire home/apt,...,0,0,0,0,0,0,1,1,1,POINT (11.244 43.782)
2,24471,https://www.airbnb.com/rooms/24471,2021-12-17,99178,2010-03-26,5,Centro Storico,43.78202,11.24399,Entire home/apt,...,0,0,0,0,0,0,1,1,1,POINT (11.244 43.782)
3,24472,https://www.airbnb.com/rooms/24472,2021-12-17,99178,2010-03-26,5,Centro Storico,43.78202,11.24399,Entire home/apt,...,0,0,0,0,0,0,1,1,1,POINT (11.244 43.782)
4,26738,https://www.airbnb.com/rooms/26738,2021-12-17,113883,2010-04-26,2,Centro Storico,43.77017,11.25754,Private room,...,0,0,0,0,0,0,0,0,1,POINT (11.2575 43.7702)


#add valid data for geom column for each listing. 
#latitude, longitude, srid=4326
#concatenate, long, lat, and srid=4326
#x=None, y=None, z=None, srid=None
#location = fromstr(f'POINT({longitude} {latitude})', srid=4326)


In [61]:
df.head()
df.shape



(10268, 51)

In [28]:
def create_geocensus_variables(ia_newvar_file_input, file_output, geodataframe):
#need the joined gdf with airbnb data for the stats on census because we use the listing row id of the airbnb file which only matches with the gdf because it has airnbb data in it... maybe not necessary these variables at all anyway...we loop through csv file with 51 airbnb lisitings...to match with census info.. 
    all = []
    with open(ia_newvar_file_input, 'r') as file:
        with open(file_output, 'w') as newfile:
            csvwriter = csv.writer(newfile,lineterminator='\n')
            csvreader = csv.reader(file)
            #extract headers
            header = next(csvreader)
            new_headers = ['census_tract_id', 'num_residents_census', 'num_households_census', 'res_buildings_census', 'num_bedrooms_census', 'num_household_renters_census', 'num_household_owners_census']
            for item in new_headers:
                header.append(item)
            all.append(header)
            for row in csvreader:
            #index row 
                listing_row_id = int(row[0])
            
                #census_tract_id_sez2011
                census_tract_id = geodataframe.loc[listing_row_id, 'SEZ2011']
                #census data points 
                columns = ['P1', 'A47', 'A46', 'PF1', 'E3', 'E27', 'SEZ2011']
                #number of residents per census tract P1
                num_residents_census = (geodataframe.loc[listing_row_id, 'P1'])
                #number of households per census tract PF1
                num_households_census = (geodataframe.loc[listing_row_id, 'PF1'])
                #number of residential buildings per census tract E3
                res_buildings_census = (geodataframe.loc[listing_row_id, 'E3'])
                # number of bedrooms per census tract E27
                num_bedrooms_census = (geodataframe.loc[listing_row_id, 'E27'])
                #number of household renters per census tract  A46 
                num_household_renters_census = (geodataframe.loc[listing_row_id, 'A46'])
                #number of household homeowners per census tract A47
                num_household_owners_census = (geodataframe.loc[listing_row_id, 'A47'])
                #adding new variable to row for each row in file
                
                new_variables = [census_tract_id, num_residents_census, num_households_census, res_buildings_census, num_bedrooms_census, num_household_renters_census, num_household_owners_census]
                for item in new_variables:
                    row.append(item)
                #adding entire row to all 
                all.append(row)
            #writing all rows to new out file
            csvwriter.writerows(all)
    return csvwriter
create_geocensus_variables('../thesis_app/Out_CSV/ia_1_27_222022_03_23_18_40_54_new_variables.csv', '../thesis_app/Out_CSV/ia_1_27_222022_03_23_18_40_54_new_variables_census.csv', census_gdf)